In [ ]:
import os
import tensorflow as tf
import pickle
    
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.123.22.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.22.106:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import re
import random
import time

print('Library versions:')

import keras
print(f'keras:{keras.__version__}')
import pandas as pd
print(f'pandas:{pd.__version__}')

import sklearn
print(f'sklearn:{sklearn.__version__}')
import nltk
print(f'nltk:{nltk.__version__}')
import numpy as np
print(f'numpy:{np.__version__}')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize

from tqdm import tqdm_notebook as tqdm

Library versions:
keras:2.7.0
pandas:1.1.5
sklearn:1.0.1
nltk:3.2.5
numpy:1.19.5


In [ ]:
MAX_VOCAB_SIZE = 2**13
MAX_MESSAGE_LEN = 30  
EMBEDDING_SIZE = 100
CONTEXT_SIZE = 100
BATCH_SIZE = 4
DROPOUT = 0.2
LEARNING_RATE=0.005

# Tokens needed for seq2seq
UNK = 0  # words that aren't found in the vocab
PAD = 1  # after message has finished, this fills all remaining vector positions
START = 2  # provided to the model at position 0 for every response predicted

# Implementaiton detail for allowing this to be run in Kaggle's notebook hardware
SUB_BATCH_SIZE = 1000

In [ ]:
tweets = pd.read_csv('/content/drive/MyDrive/Saswat/archive (5)/twcs/twcs.csv')

first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]

inbounds_and_outbounds = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id').sample(frac=1)

In [ ]:
# Filter to only outbound replies (from spotifycares)
x = inbounds_and_outbounds[inbounds_and_outbounds.author_id_y == 'SpotifyCares']
inbounds_and_outbounds = x[inbounds_and_outbounds.inbound_y ^ True]

tqdm().pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


0it [00:00, ?it/s]

In [ ]:
print(f'Data shape: {inbounds_and_outbounds.shape}')

Data shape: (26940, 14)


In [ ]:
# Replace anonymized screen names with common token @__sn__
def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return '@__sn__'

sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')

x_text = inbounds_and_outbounds.text_x.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))
y_text = inbounds_and_outbounds.text_y.progress_apply(lambda txt: sn_re.sub(sn_replace, txt))

  0%|          | 0/26940 [00:00<?, ?it/s]

  0%|          | 0/26940 [00:00<?, ?it/s]

In [ ]:
count_vec = CountVectorizer(tokenizer=casual_tokenize, max_features=MAX_VOCAB_SIZE - 3)
print("Fitting CountVectorizer on X and Y text data...")
count_vec.fit((x_text + y_text))
analyzer = count_vec.build_analyzer()
vocab = {k: v + 3 for k, v in count_vec.vocabulary_.items()}
vocab['__unk__'] = UNK
vocab['__pad__'] = PAD
vocab['__start__'] = START

# Used to turn seq2seq predictions into human readable strings
reverse_vocab = {v: k for k, v in vocab.items()}
print(f"Learned vocab of {len(vocab)} items.")

Fitting CountVectorizer on X and Y text data...


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


Learned vocab of 8192 items.


In [ ]:
def to_word_idx(sentence):
    full_length = [vocab.get(tok, UNK) for tok in analyzer(sentence)] + [PAD] * MAX_MESSAGE_LEN
    return full_length[:MAX_MESSAGE_LEN]

def from_word_idx(word_idxs):
    return ' '.join(reverse_vocab[idx] for idx in word_idxs if idx != PAD).strip()

In [ ]:
x_text.head().apply(to_word_idx).apply(from_word_idx)
x = pd.np.vstack(x_text.progress_apply(to_word_idx).values)
y = pd.np.vstack(y_text.progress_apply(to_word_idx).values)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


  0%|          | 0/26940 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/26940 [00:00<?, ?it/s]

In [ ]:
all_idx = list(range(len(x)))
train_idx = set(random.sample(all_idx, int(0.8 * len(all_idx))))
test_idx = {idx for idx in all_idx if idx not in train_idx}

train_x = x[list(train_idx)]
test_x = x[list(test_idx)]
train_y = y[list(train_idx)]
test_y = y[list(test_idx)]

assert train_x.shape == train_y.shape
assert test_x.shape == test_y.shape

print(f'Training data of shape {train_x.shape} and test data of shape {test_x.shape}.')

Training data of shape (21552, 30) and test data of shape (5388, 30).


In [ ]:
from keras.models import Model
from keras.optimizers import adam_v2


from keras.layers import Dense, Input, LSTM, Dropout, Embedding, RepeatVector, concatenate, \
    TimeDistributed
from keras.utils import np_utils

import tensorflow as tf
import os

In [ ]:
# use TPU
def create_model():
    shared_embedding = Embedding(
        output_dim=EMBEDDING_SIZE,
        input_dim=MAX_VOCAB_SIZE,
        input_length=MAX_MESSAGE_LEN,
        name='embedding',
    )
    
    # ENCODER
    
    encoder_input = Input(
        shape=(MAX_MESSAGE_LEN,),
        dtype='int32',
        name='encoder_input',
    )
    
    embedded_input = shared_embedding(encoder_input)
    

    # input sequence provided.
    encoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='encoder',
        dropout=DROPOUT
    )
    
    context = RepeatVector(MAX_MESSAGE_LEN)(encoder_rnn(embedded_input))
    
    # DECODER
    
    last_word_input = Input(
        shape=(MAX_MESSAGE_LEN, ),
        dtype='int32',
        name='last_word_input',
    )
    
    embedded_last_word = shared_embedding(last_word_input)
    
    decoder_input = concatenate([embedded_last_word, context], axis=2)
    
    decoder_rnn = LSTM(
        CONTEXT_SIZE,
        name='decoder',
        return_sequences=True,
        dropout=DROPOUT
    )
    
    decoder_output = decoder_rnn(decoder_input)
    
    # TimeDistributed allows the dense layer to be applied to each decoder output per timestep
    next_word_dense = TimeDistributed(
        Dense(int(MAX_VOCAB_SIZE / 2), activation='relu'),
        name='next_word_dense',
    )(decoder_output)
    
    next_word = TimeDistributed(
        Dense(MAX_VOCAB_SIZE, activation='softmax'),
        name='next_word_softmax'
    )(next_word_dense)
    
    return Model(inputs=[encoder_input, last_word_input], outputs=[next_word])


optimizer = tf.optimizers.Adam(lr=LEARNING_RATE, clipvalue=5.0)

# use TPU
with tpu_strategy.scope():
    
    s2s_model = create_model()
    s2s_model.compile(optimizer='adam', loss='categorical_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
def add_start_token(y_array):
    """ Adds the start token to vectors.  Used for training data. """
    return np.hstack([
        START * np.ones((len(y_array), 1)),
        y_array[:, :-1],
    ])

def binarize_labels(labels):
    """ Helper function that turns integer word indexes into sparse binary matrices for 
        the expected model output.
    """
    return np.array([np_utils.to_categorical(row, num_classes=MAX_VOCAB_SIZE)
                     for row in labels])

In [ ]:
def respond_to(model, text):
    """ Helper function that takes a text input and provides a text output. """
    input_y = add_start_token(PAD * np.ones((1, MAX_MESSAGE_LEN)))
    idxs = np.array(to_word_idx(text)).reshape((1, MAX_MESSAGE_LEN))
    for position in range(MAX_MESSAGE_LEN - 1):
        prediction = model.predict([idxs, input_y]).argmax(axis=2)[0]
        input_y[:,position + 1] = prediction[position]
    return from_word_idx(model.predict([idxs, input_y]).argmax(axis=2)[0])

In [ ]:
def train_mini_epoch(model, start_idx, end_idx, epoch):
    """ Batching seems necessary in Kaggle Jupyter Notebook environments, since
        `model.fit` seems to freeze on larger batches (somewhere 1k-10k).
    """
    b_train_y = binarize_labels(train_y[start_idx:end_idx])
    input_train_y = add_start_token(train_y[start_idx:end_idx])
    
    model.fit(
        [train_x[start_idx:end_idx], input_train_y], 
        b_train_y,
        epochs=1,
        batch_size=BATCH_SIZE,
    )
    
    rand_idx = random.sample(list(range(len(test_x))), SUB_BATCH_SIZE)
    
    
    if epoch == 9:
        print('Test results:', model.evaluate(
            [test_x[rand_idx], add_start_token(test_y[rand_idx])],
            binarize_labels(test_y[rand_idx])
        ))

In [ ]:
training_time_limit = 360 * 600  # seconds (notebooks terminate after 10 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(3):
        print(f'Training in epoch {epoch}...')

            
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE, epoch)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

Training in epoch 0...
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


138/138 [==============================] - 4s 28ms/step - loss: 1.2436
Training in epoch 1...
138/138 [==============================] - 4s 30ms/step - loss: 1.0363
Training in epoch 2...
138/138 [==============================] - 4s 29ms/step - loss: 0.9245


In [ ]:
s2s_model.save(filepath='/content/drive/MyDrive/Saswat/model.h5')

In [ ]:
# If want to train more
s2s_model = keras.models.load_model('/content/drive/MyDrive/Saswat/model.h5')

In [ ]:
training_time_limit = 360 * 600  # seconds (notebooks terminate after 10 hour)
start_time = time.time()
stop_after = start_time + training_time_limit

class TimesUpInterrupt(Exception):
    pass

try:
    for epoch in range(7):
        print(f'Training in epoch {epoch}...')

            
        for start_idx in range(0, len(train_x), SUB_BATCH_SIZE):
            train_mini_epoch(s2s_model, start_idx, start_idx + SUB_BATCH_SIZE, epoch)
            if time.time() > stop_after:
                raise TimesUpInterrupt
except KeyboardInterrupt:
    print("Halting training from keyboard interrupt.")
except TimesUpInterrupt:
    print(f"Halting after {time.time() - start_time} seconds spent training.")

Training in epoch 0...
138/138 [==============================] - 11s 77ms/step - loss: 0.9240
Training in epoch 1...
138/138 [==============================] - 11s 79ms/step - loss: 0.8589
Training in epoch 2...
138/138 [==============================] - 11s 76ms/step - loss: 0.8073
Training in epoch 3...
138/138 [==============================] - 11s 79ms/step - loss: 0.7637
Training in epoch 4...
138/138 [==============================] - 11s 78ms/step - loss: 0.7280
Training in epoch 5...
138/138 [==============================] - 11s 81ms/step - loss: 0.6968
Training in epoch 6...
138/138 [==============================] - 11s 80ms/step - loss: 0.6697


In [ ]:
s2s_model.save(filepath='/content/drive/MyDrive/Saswat/model.h5')
# Model trained for n more epochs 

In [ ]:
w = keras.models.load_model('/content/drive/MyDrive/Saswat/model.h5')

In [ ]:
test = list(inbounds_and_outbounds.head(150)['text_x'].values)[100:]

In [ ]:
for i in test:
    print(f'Query: {i}')
    print(f'Response: {respond_to(w, i)}')

Query: @SpotifyCares hello! when is support coming for the iPhone X? your app isn’t looking very pretty ☹️ https://t.co/dfuO9PzFZF
Response: @__sn__ hey there ! we ’ d love to have all of their stuff available , but we have some info about content here : https://t.co/0i8gpimuda / ar
Query: @115888 when will you extent yourself to South Africa Please 😩😩🙏🏽🙏🏽🇿🇦
Response: @__sn__ hey ! we're launching in new countries as often as possible . be sure to add your email at https://t.co/x8dvsx9qgm / cx
Query: Hi @SpotifyCares, I want to change the date that money leaves my account for premium, is this possible or do I have to cancel and renew again??
Response: @__sn__ hey there ! can you dm us your account's email address ? we'll take a look backstage / rb https://t.co/ldfdzrinat
Query: @SpotifyCares redeem code for premium family not working. Have cancelled and resent but still says code does not seem to be valid when redeeming. How can I fix this??
Response: @__sn__ hey there ! can you dm us 

In [ ]:
a_list = [
    
" check the dms please",
" can you just add a recorded version of 'Shock the Monkey' by Peter Gabriel?",
" is there any way that I can block all boogie with a hoodie songs? #protectlilbatallcosts",
" Client keeps shutting down and relaunching in mac (10.13.1), at about 2 times per second. Seem to write a copy of itself each time (!!!) Tried solution in official support site; it doesn't work... :/",
" CHECK YA DMSSSSSSS",
" I was wondering how often do I have to renew my student discount? The last time I renewed it was in July and I have been charged to the full $9.99 montly for the past 3 months including this month so that means I renewed it and it was $4.99 for 1 month.",
" I loaded mp3s onto my laptop, added to spotify, made playlist. Why cant i download playlist to my phone spotify app?",
" my iPhone app deleted my entire downloaded library for the 12th time!!! tell them there should be keyboard shortcuts for everything!",
" sent a DM :/",
" we have Shania problem!!!! Her alblum, 'Come on over', only have the uts inferior version",
" what is going on!!!! I got logged out and lost all my songs",
" When will Spotify be launched in India? ;))) ",
" when will you be supported in israel? ;/",
" Hey, I would love to be able to hide the “Browse” page, so I see just my music when I open Spotify, not ads for bands I hate and “funny” texts. ",
" Hi is it possible to upload Rivermaya’s old song “You’ll Be Safe Here”?",
" I don’t understand why my scratches at times ",
" Jadah apa tolak duit tapi still ada iklan 'langkawi' ",
" stop playing and bring back the touch preview option!!! ",
" can we get more Ed Sheeren songs in spotify ???",
" Why is Motty Steinmetz' music not avalible on Spotify? Get in contact with his team and work to fix that s"

]

In [ ]:
for i in a_list:
    print(f'Query: {i}')
    print(f'Response: {respond_to(w, i)}')
    print('\n')

Query:  check the dms please
Response: @__sn__ hey there ! can you dm us your account's email address ? we'll take a look / rb https://t.co/ldfdzrinat


Query:  can you just add a recorded version of 'Shock the Monkey' by Peter Gabriel?
Response: @__sn__ hey there ! we ’ d love to have all of their stuff available , but we have some info about content here : https://t.co/0i8gpimuda / ar


Query:  is there any way that I can block all boogie with a hoodie songs? #protectlilbatallcosts
Response: @__sn__ hey there ! we ’ d love to have all of their stuff available , but we have some info about content here : https://t.co/0i8gpimuda / ar


Query:  Client keeps shutting down and relaunching in mac (10.13.1), at about 2 times per second. Seem to write a copy of itself each time (!!!) Tried solution in official support site; it doesn't work... :/
Response: @__sn__ hey there ! can you let us know the device , operating system , and spotify version you're using ? we'll see what we can suggest /